In [8]:
### Load packages and data files
import csv
import glob
from pprint import pprint 
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from dateutil import parser
import numpy as np
import math
import random
from operator import add

%matplotlib inline

In [9]:
## Function to Read all text files and Store in a List
def turnstile_read_csv(csv_files):
    dict_MTA= defaultdict(list)
    MTA = [] 
    for filename in glob.glob(csv_files):
        file = open(filename, 'r')
        reader = csv.reader(file)
        header = reader.next()
        MTA = list(reader)
        for turnstile in MTA:
            turnstile[-1].strip()     # Strip does not seem to work, fix later
            turnstile_key = tuple(turnstile[0:5])
            dict_MTA[turnstile_key].append(turnstile[5:])
        file.close()
    return dict_MTA


In [10]:
## Read all text files and Store in a List
dict_MTA = turnstile_read_csv("turnstile_*.txt")
pprint ({k: dict_MTA[k][:2] for k in dict_MTA.keys()[:2]})

{('JFK03', 'R536', '00-00-02', 'JFK JAMAICA CT1', 'E'): [['IND',
                                                          '06/04/2016',
                                                          '01:00:00',
                                                          'REGULAR',
                                                          '0000087367',
                                                          '0000079390                                       '],
                                                         ['IND',
                                                          '06/04/2016',
                                                          '05:00:00',
                                                          'REGULAR',
                                                          '0000087372',
                                                          '0000079392                                       ']],
 ('R238A', 'R046', '02-03-02', 'GRD CNTRL-42 ST', '4567S'): [['IRT',
          

## Get counts per interval

In [13]:
from datetime import datetime, date
from dateutil import parser
def time_series(dict_MTA):
    dict_MTA_ts = {}
    for key in dict_MTA.keys():
        dict_MTA_ts[key] = []
        for i in range(0, len(dict_MTA[key])):
            time = dict_MTA[key][i][1] + " " + dict_MTA[key][i][2]
            four_hr_ct = int(dict_MTA[key][i][4])- int(dict_MTA[key][i-1][4])
            #print "Count: %d" %four_hr_ct
            #gets entries and exits for each station
            four_hr_ct_entry = int(dict_MTA[key][i][4])- int(dict_MTA[key][i-1][4])
            four_hr_ct_exit = int(dict_MTA[key][i][5]) - int(dict_MTA[key][i-1][5])
            #checks that exits and entries are not negative and adds accordingly
            if four_hr_ct_entry < 0 or four_hr_ct_entry > 4*60*20 and four_hr_ct_exit < 0 or four_hr_ct_exit > 4*60*20:
                dict_MTA_ts[key].append([parser.parse(time), None])
            elif four_hr_ct_entry < 0 or four_hr_ct_entry > 4*60*20:
                dict_MTA_ts[key].append([parser.parse(time), four_hr_ct_exit])
            elif four_hr_ct_exit < 0 or four_hr_ct_exit > 4*60*20:
                dict_MTA_ts[key].append([parser.parse(time), four_hr_ct_entry])
            else:
                dict_MTA_ts[key].append([parser.parse(time), four_hr_ct_entry + four_hr_ct_exit])
    return dict_MTA_ts


In [12]:
small = dict((k, dict_MTA[k]) for k in (('N134', 'R385', '00-00-00', 'ROCKAWAY BLVD', 'A'), ('A006', 'R079' , '00-00-01' , '5 AV/59 ST','NQR')))
ts_small = time_series(small)
print ts_small[('N134', 'R385', '00-00-00', 'ROCKAWAY BLVD','A')][:10]
print ts_small[('A006', 'R079' , '00-00-01' , '5 AV/59 ST','NQR')][:10]

[[datetime.datetime(2016, 6, 4, 1, 0), None], [datetime.datetime(2016, 6, 4, 5, 0), 33], [datetime.datetime(2016, 6, 4, 9, 0), 226], [datetime.datetime(2016, 6, 4, 13, 0), 208], [datetime.datetime(2016, 6, 4, 17, 0), 261], [datetime.datetime(2016, 6, 4, 21, 0), 232], [datetime.datetime(2016, 6, 5, 1, 0), 125], [datetime.datetime(2016, 6, 5, 5, 0), 30], [datetime.datetime(2016, 6, 5, 9, 0), 140], [datetime.datetime(2016, 6, 5, 13, 0), 166]]
[[datetime.datetime(2016, 6, 4, 2, 0), None], [datetime.datetime(2016, 6, 4, 6, 0), 16], [datetime.datetime(2016, 6, 4, 10, 0), 146], [datetime.datetime(2016, 6, 4, 14, 0), 382], [datetime.datetime(2016, 6, 4, 18, 0), 592], [datetime.datetime(2016, 6, 4, 22, 0), 500], [datetime.datetime(2016, 6, 5, 2, 0), 110], [datetime.datetime(2016, 6, 5, 6, 0), 10], [datetime.datetime(2016, 6, 5, 10, 0), 112], [datetime.datetime(2016, 6, 5, 14, 0), 385]]


In [7]:
# Run all time series
dict_MTA_ts = time_series(dict_MTA)

KeyboardInterrupt: 

### Combine stations and lines and attempt to adjust for incongruent line-station combos

In [ ]:
# Fix some naming of station-lines, more needed on future steps
def convert(station, line):
    try:
        if station == "34 ST-PENN STA": return "123ACE"
        return {'ACJZ2345': '2345ACJZ', 'ACENQRS1237': '1237ACENQRS','AC1' : '1AC', 
         'JZ456' : '456JZ', 'BDNQR2345' : '2345BDNQR', 'ABCD1' : '1ABCD', 
         'R2345' : '2345R', 'LNQR456' : '456LNQR', 'BD4' : '4BD',
         'ACENGRS1237': '1237ACENQRS'}[line]
    except:
        return line

## Merge per station per morning of the week

In [ ]:
def combine_into_stations(MTA):
    dict_station = defaultdict(list)
    for key in MTA:
        station_key = key[3] + " _ " + convert(key[3], key[4])
        #print station_key
        for interval in MTA[key]:
            #Eprint "Interval: ",
            #print interval 
            #print 14 >= interval[0].hour and interval[0].hour >= 8
            if(14 >= interval[0].hour and interval[0].hour >= 8):
                dict_station[station_key].append([interval[0].weekday(), interval[0].hour, interval[1]])
            #print dict_station
    return dict_station

In [ ]:
station_small = combine_into_stations(ts_small)
print station_small   # for each day of the month [day f the week, time, count]

In [ ]:
# Applying fn to merge per morning of week by station
dict_stations = combine_into_stations(dict_MTA_ts)
print dict_stations['5 AV/59 ST _ NQR']

## Creates dictionary of format station:[avg count from Mon AM, for Tues AM, ...]

In [ ]:
def days_of_week(dict_station):
    dict_by_day = defaultdict(list)
    for key in dict_station:
        day_counter = [0,0,0,0,0,0,0]
        dict_by_day[key] = [None,None,None,None,None,None,None]
        #print len(dict_station[key])
        
        for entry in dict_station[key]:
            day_counter[entry[0]]+=1
            #print dict_by_day[key]
            #print entry[2]
            if dict_by_day[key][entry[0]] == None and entry[2] != None:
                dict_by_day[key][entry[0]] = float(entry[2])
            elif entry[2] != None:
                dict_by_day[key][entry[0]] += float(entry[2])
        for i in range(0,7):
            dict_by_day[key][i] = dict_by_day[key][i]/(float(day_counter[i])/2)  
    #print day_counter
    
    return dict_by_day

In [ ]:
days_small = days_of_week(station_small)
print days_small

In [ ]:
dict_by_days = days_of_week(dict_stations)
#print len(dict_by_days)
#print dict_by_days

## Prints out all stations to csv

In [ ]:
#print dict_by_days.keys()
#import csv
with open('stations.csv','w') as file:
    writer = csv.writer(file, lineterminator=',')
    for i in dict_by_days.keys():
        writer.writerow([i])

## Add by Neighborhood

In [ ]:
neighbor_key=defaultdict()

with open ('Neighborhoods.csv') as csv_file:
    data=csv.reader(csv_file)
    for line in data:
        neighbor_key[line[0]]=line[1]

In [ ]:
all_stations=neighbor_key.keys()
averages=dict_by_days    #### CHANGE OF VAR NAME

In [ ]:
averages['125 ST _ 23']

In [ ]:
neighbor_influx=defaultdict(list)

for station in neighbor_key.keys():
    hood = neighbor_key[station]
    if neighbor_influx[hood]:
        neighbor_influx[hood]= map(add, neighbor_influx[hood],averages[station])   ## Sum of averages!!!! 
    else:
        neighbor_influx[hood]=averages[station]

In [ ]:
neighbor_influx['CHELSEA']